In [ ]:
# prompt: link to drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import shutil
import glob
from google.colab import drive

In [ ]:
%cd /content/drive/MyDrive/lane_bien/data
!unzip /content/drive/MyDrive/lane_bien/lanevippro.zip

/content/drive/MyDrive/lane_bien/data
Archive:  /content/drive/MyDrive/lane_bien/lanevippro.zip
   creating: lanevippro/
  inflating: lanevippro/data.yaml    
  inflating: lanevippro/README.dataset.txt  
  inflating: lanevippro/README.roboflow.txt  
   creating: lanevippro/test/
   creating: lanevippro/test/images/
  inflating: lanevippro/test/images/img_20251105_075130_0005_jpg.rf.40bf83c2af7dbfb069a7844ce6daeedd.jpg  
  inflating: lanevippro/test/images/img_20251105_075137_0018_jpg.rf.49fa52f3b0f5e8605a5520102be574c2.jpg  
  inflating: lanevippro/test/images/img_20251105_075147_0039_jpg.rf.d645fcce4fb1a04b4085fb32905c77ef.jpg  
  inflating: lanevippro/test/images/img_20251105_075150_0045_jpg.rf.43544ea8afa3436c4b698e769ba6e488.jpg  
  inflating: lanevippro/test/images/img_20251105_075154_0053_jpg.rf.32e174b57a460584023306eb682acf5b.jpg  
  inflating: lanevippro/test/images/img_20251105_075156_0057_jpg.rf.7c253fcb1e80c36fe1df76d579b1e73d.jpg  
  inflating: lanevippro/test/images/img_2

In [ ]:
%cd /content/drive/MyDrive/lane_bien/data
!unzip /content/drive/MyDrive/lane_bien/bienbao.zip

/content/drive/MyDrive/lane_bien/data
Archive:  /content/drive/MyDrive/lane_bien/bienbao.zip
   creating: bienbao/
  inflating: bienbao/data.yaml       
  inflating: bienbao/README.dataset.txt  
  inflating: bienbao/README.roboflow.txt  
   creating: bienbao/test/
   creating: bienbao/test/images/
  inflating: bienbao/test/images/traffic-108-_jpg.rf.fdda27b8ce061403cc4a27c5bed74978.jpg  
  inflating: bienbao/test/images/traffic-17-_jpg.rf.7c4c6ad956a019d44f4904b2b38c591b.jpg  
  inflating: bienbao/test/images/traffic-23-_jpg.rf.582f46202f2a058a32c70bf49ee44f52.jpg  
  inflating: bienbao/test/images/traffic-27-_jpg.rf.14969fe397fd9640b3d81c0838da472b.jpg  
  inflating: bienbao/test/images/traffic-46-_jpg.rf.8d181763c0460f135bda786682fa22b1.jpg  
  inflating: bienbao/test/images/traffic-51-_jpg.rf.3872db41672abcfdd0b57237fb620e49.jpg  
  inflating: bienbao/test/images/traffic-59-_jpg.rf.681d007dcc6cd4703aeb4b3ebeaec724.jpg  
  inflating: bienbao/test/images/traffic-65-_jpg.rf.d87a42538de

In [ ]:
import os, shutil, yaml
from tqdm import tqdm

lane_dir = "/content/drive/MyDrive/lane_bien/data/lanevippro"
sign_dir = "/content/drive/MyDrive/lane_bien/data/bienbao"
merged_dir = "/content/drive/MyDrive/lane_bien/Merged"

os.makedirs(f"{merged_dir}/train/images", exist_ok=True)
os.makedirs(f"{merged_dir}/train/labels", exist_ok=True)
os.makedirs(f"{merged_dir}/val/images", exist_ok=True)
os.makedirs(f"{merged_dir}/val/labels", exist_ok=True)

# đọc YAML
with open(f"{lane_dir}/data.yaml", 'r') as f:
    data_lane = yaml.safe_load(f)
with open(f"{sign_dir}/data.yaml", 'r') as f:
    data_sign = yaml.safe_load(f)

# gộp danh sách class
names = list(dict.fromkeys(data_lane['names'] + data_sign['names']))
print("✅ Danh sách class sau khi gộp:")
for i, name in enumerate(names):
    print(f"{i}: {name}")

# hàm gộp dữ liệu
def merge_split(split_src, split_dst, src_dir, offset):
    img_src = os.path.join(src_dir, split_src, "images")
    lbl_src = os.path.join(src_dir, split_src, "labels")
    img_dst = os.path.join(merged_dir, split_dst, "images")
    lbl_dst = os.path.join(merged_dir, split_dst, "labels")

    os.makedirs(img_dst, exist_ok=True)
    os.makedirs(lbl_dst, exist_ok=True)

    # copy ảnh
    for img_file in tqdm(os.listdir(img_src), desc=f"Copy {split_dst} images from {src_dir}"):
        shutil.copy(os.path.join(img_src, img_file), os.path.join(img_dst, img_file))

    # copy label và sửa class id
    for lbl_file in tqdm(os.listdir(lbl_src), desc=f"Copy {split_dst} labels from {src_dir}"):
        src_path = os.path.join(lbl_src, lbl_file)
        dst_path = os.path.join(lbl_dst, lbl_file)
        with open(src_path, 'r') as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls = int(parts[0]) + offset
                new_lines.append(" ".join([str(cls)] + parts[1:]) + "\n")

        with open(dst_path, 'w') as f:
            f.writelines(new_lines)

# gộp train và valid (chuyển valid → val)
merge_split("train", "train", lane_dir, 0)
merge_split("valid", "val", lane_dir, 0)
merge_split("train", "train", sign_dir, len(data_lane['names']))
merge_split("valid", "val", sign_dir, len(data_lane['names']))

# tạo YAML cho dataset gộp
merged_yaml = {
    'train': f"{merged_dir}/train/images",
    'val': f"{merged_dir}/val/images",
    'nc': len(names),
    'names': names
}

with open(f"{merged_dir}/data.yaml", 'w') as f:
    yaml.dump(merged_yaml, f, sort_keys=False)

print("\n🎉 Gộp hoàn tất! File YAML lưu tại:", f"{merged_dir}/data.yaml")


✅ Danh sách class sau khi gộp:
0: lane
1: phai
2: trai
3: speed20
4: speed30
5: speed50
6: stop


Copy train images from /content/drive/MyDrive/lane_bien/data/lanevippro: 100%|██████████| 279/279 [00:04<00:00, 57.48it/s]
Copy train labels from /content/drive/MyDrive/lane_bien/data/lanevippro: 100%|██████████| 279/279 [00:05<00:00, 55.13it/s]
Copy val images from /content/drive/MyDrive/lane_bien/data/lanevippro: 100%|██████████| 20/20 [00:00<00:00, 66.16it/s]
Copy val labels from /content/drive/MyDrive/lane_bien/data/lanevippro: 100%|██████████| 20/20 [00:00<00:00, 66.16it/s]
Copy train images from /content/drive/MyDrive/lane_bien/data/bienbao: 100%|██████████| 258/258 [00:04<00:00, 52.61it/s]
Copy train labels from /content/drive/MyDrive/lane_bien/data/bienbao: 100%|██████████| 258/258 [00:04<00:00, 52.49it/s]
Copy val images from /content/drive/MyDrive/lane_bien/data/bienbao: 100%|██████████| 15/15 [00:00<00:00, 64.98it/s]
Copy val labels from /content/drive/MyDrive/lane_bien/data/bienbao: 100%|██████████| 15/15 [00:00<00:00, 68.80it/s]


🎉 Gộp hoàn tất! File YAML lưu tại: /content/drive/MyDrive/lane_bien/Merged/data.yaml


In [ ]:
%cat /content/drive/MyDrive/lane_bien/Merged/.yaml


train: /content/drive/MyDrive/lane_bien/Merged/train/images
val: /content/drive/MyDrive/lane_bien/Merged/val/images
nc: 7
names:
- lane
- phai
- trai
- speed20
- speed30
- speed50
- stop


In [ ]:
# clone YOLOv11
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -q ultralytics


Cloning into 'ultralytics'...
remote: Enumerating objects: 72529, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 72529 (delta 153), reused 80 (delta 80), pack-reused 72288 (from 2)
Receiving objects: 100% (72529/72529), 39.07 MiB | 15.53 MiB/s, done.
Resolving deltas: 100% (54373/54373), done.
Updating files: 100% (787/787), done.
/content/drive/MyDrive/lane_bien/YOLO_train/ultralytics
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.9 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/lane_bien/YOLO_train
!yolo train model=yolo11n.pt data=/content/drive/MyDrive/lane_bien/Merged/data.yaml epochs=100 imgsz=640 patience= 10


/content/drive/MyDrive/lane_bien/YOLO_train
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/lane_bien/Merged/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.